God bless these people for [this stack overflow topic](https://stackoverflow.com/questions/14692690/access-nested-dictionary-items-via-a-list-of-keys). I unshamefully stole the code from there.

In [103]:
from functools import reduce  # forward compatibility for Python 3
import operator

def setInDict(dataDict, mapList, value):
    getFromDict(dataDict, mapList[:-1])[mapList[-1]] = value

def getFromDict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

In order to process input to get a tree representation into a dict

In [104]:
# Use of generator to avoid filling memory with data we don't need (after the start signal)
def reader(filename):
    with open(filename) as f:
        while True:
            # read next line
            line = f.readline()
            if line:
                # return line
                yield line
            else:
                return

command_number = 0
output_number = 0

tree_representation = {"/" : {}}
working_directory = []

for line in reader('day7Input.txt'):
    
    if line[0:1] == "$":
        command_number += 1
        print(f"Command {command_number}")
        output_number = 0

    else :
        output_number += 1
        print(f"        output {output_number}")
        size, filename = line.split()
        if size == "dir":
            # Adding directory to tree
            print(f"            adding directory {filename} : {'{}'}")

            # getting the current value of the key
            current_value = getFromDict(tree_representation, working_directory)

            # if the key is not in the dictionary, add it
            if filename not in current_value:
                current_value[filename] = {}

            setInDict(tree_representation, working_directory, current_value)
        else :
            # Adding a file to tree
            print(f"            adding file {filename} : {size}")

            # getting the current value of the key
            current_value = getFromDict(tree_representation, working_directory)

            # if the key is not in the dictionary, add it
            if filename not in current_value:
                current_value[filename] = int(size)
            
            setInDict(tree_representation, working_directory, current_value)

    # If cd then change working directory
    if line[0:4] == "$ cd":
        directory = line[5:-1]
        print(f"    Changing directory to {directory}")
        value_of_working_directory = getFromDict(tree_representation, working_directory)
        keys = value_of_working_directory.keys()
        if directory in keys:
            working_directory.append(str(directory).replace('\'', '"'))
        elif directory == "..":
            # Remove last element in working_directory
            working_directory.pop()
        else :
            print(f"        Directory {directory} not found in path {working_directory}")

    # If ls, then register files and directory
    if line[0:4] == "$ ls":
        print(f"    Listing files and directories in {working_directory}")
        
print(tree_representation)

Command 1
    Changing directory to /
Command 2
    Listing files and directories in ['/']
        output 1
            adding directory dfmhjhd : {}
        output 2
            adding file ghpqs : 307728
        output 3
            adding directory hztjntff : {}
        output 4
            adding directory rvstq : {}
        output 5
            adding directory sjt : {}
        output 6
            adding file whhj.pqt : 120579
        output 7
            adding directory wrmm : {}
Command 3
    Changing directory to dfmhjhd
Command 4
    Listing files and directories in ['/', 'dfmhjhd']
        output 1
            adding file ngtqtf : 301486
        output 2
            adding file wfgqtw.sqr : 13488
Command 5
    Changing directory to ..
Command 6
    Changing directory to hztjntff
Command 7
    Listing files and directories in ['/', 'hztjntff']
        output 1
            adding directory cwsf : {}
        output 2
            adding file ftq.cjn : 288227
        output 3
  

Now the processing (didn't had time to make it clear and commented)

In [105]:
def get_keys(dictionary):
    result = []
    for key, value in dictionary.items():
        if type(value) is dict:
            new_keys = get_keys(value)
            result.append(key)
            for innerkey in new_keys:
                result.append(f'{key}\\{innerkey}')
        else:
            result.append(key)
    return result

light_directories_total_weight = 0
directories = {}
possible_space_saving = []

for key in get_keys(tree_representation):
    path = key.split('\\')

    if type(getFromDict(tree_representation, path)) != int:
        size_directory = 0
        value = getFromDict(tree_representation, path)
        keys = get_keys(value)
        for key in keys:
            path_key = path + [key.split('\\')][0]
            value = getFromDict(tree_representation, path_key)
            if type(value) == int:
                size_directory += value
        
        directories[str(path)] = size_directory
        # print(f"    Size of directory {path} : {size_directory}")
        if size_directory < 100000:
            light_directories_total_weight += size_directory
        if size_directory > 30000000:
            possible_space_saving.append(size_directory)
                

print(f"Total weight of light directories : {light_directories_total_weight} bytes")

Total weight of light directories : 1743217 bytes


In [106]:
# in directory dictionnary, get the index of the directory with the biggest size
biggest_directory = max(directories, key=directories.get)

left_space = 70000000 - directories[biggest_directory]

possible_to_delete_folders = []

for value in directories.values():
    if left_space + value > 30000000:
        possible_to_delete_folders.append(value)

print(f"Best folder to delete weights {min(possible_to_delete_folders)} bytes")

Best folder to delete weights 8319096 bytes
